
# 🧪 ImpliRet Evaluation Notebook

This tutorial lets you:

1. **Load** any of the six ImpliRet subsets.
2. **Index & evaluate** the built‑in retrievers (BM25, ColBERT‑v2, Contriever, Dragon+, HippoRAG 2, ReasonIR‑8B).
3. **Run long‑context or RAG‑style readers** (Llama‑3, GPT‑4.1, etc.) against the retrieved docs.
4. **Export** metrics to `metrics/latest.json` so the README badges stay in sync.

> **Tip**: Run on ⚡ Colab GPU for fastest turnaround.


In [ ]:
# @title Install dependencies (≈ 1 min)
# collapse-hide
DEVICE = "cuda"  #@param ["cuda", "cpu"]
!pip -q install -r https://raw.githubusercontent.com/ZeinabTaghavi/ImpliRet/main/requirements.txt

import os, json, pathlib, itertools, pprint, time


## 1 · Load a subset

In [ ]:
from datasets import load_dataset

SPLIT = "arithmetic"       #@param ["arithmetic", "wknow", "temporal"]
STYLE = "multispeaker"     #@param ["multispeaker", "unispeaker"]

ds = load_dataset("zeinabTaghavi/ImpliRet", name=STYLE, split=SPLIT)
print(f"Loaded {len(ds):,} examples  |  Columns → {list(ds.features.keys())}")
print("\nSample question →", ds[0]["question"])
print("Implicit document snippet →", ds[0]["pos_document"][:200], "...")

## 2 · Run a single retriever

In [ ]:
from Retrieval.retrieve_indexing import index_and_save
from Retrieval.reporting import evaluate_run, save_metrics_table

OUTPUT = pathlib.Path("Retrieval/results")
OUTPUT.mkdir(parents=True, exist_ok=True)

RETRIEVER = "bm25"      #@param ["bm25", "colbert", "contriever", "dragon_plus", "hipporag2", "reasonir8b"]

run_file = index_and_save(
    output_folder=str(OUTPUT),
    category=SPLIT,
    discourse=STYLE,
    retriever_name=RETRIEVER,
    device=DEVICE
)

metrics = evaluate_run(run_file)
pprint.pp(metrics)

## 3 · Benchmark all built‑ins (optional, takes time)

In [ ]:
ALL_RETRIEVERS = ["bm25", "colbert", "contriever", "dragon_plus", "hipporag2", "reasonir8b"]
table = {}

for r in ALL_RETRIEVERS:
    run_file = index_and_save(str(OUTPUT), SPLIT, STYLE, r, DEVICE)
    table[r] = evaluate_run(run_file)["ndcg@10"]
    
table

### Visualise nDCG@10

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,3))
plt.bar(table.keys(), table.values())
plt.ylabel("nDCG@10 ↑")
plt.xticks(rotation=45, ha="right")
plt.title(f"{STYLE}-{SPLIT}")
plt.show()

## 4 · RAG / Long‑context Reader

In [ ]:
from RAG_Style.scripts.sync.sync_run_tests import run_single_experiment
CONFIG = "RAG_Style/experiment_configs/bm/A_Multi_llama_bm_10.yaml"

result_path = run_single_experiment(CONFIG, hf_token=os.getenv("HF_TOKEN", ""))
print("Saved generations →", result_path)

### Compute ROUGE‑1 recall

In [ ]:
from RAG_Style.scripts.reporting import rouge_report

rouge_metrics = rouge_report(result_path)
print(rouge_metrics)

## 5 · Export metrics

In [ ]:
latest = {
    "retrieval": {"ndcg10": {"avg": sum(table.values())/len(table)}},
    "rag": {"rouge1": {"avg": rouge_metrics["rouge1"]}}
}
with open("metrics/latest.json", "w") as f:
    json.dump(latest, f, indent=2)
print("Wrote metrics/latest.json")